# Here is the code URL - https://github.com/aelassas/microservices

# Env Setup 
https://builds.dotnet.microsoft.com/dotnet/Runtime/8.0.16/dotnet-runtime-8.0.16-win-x64.exe
https://builds.dotnet.microsoft.com/dotnet/aspnetcore/Runtime/8.0.16/aspnetcore-runtime-8.0.16-win-x64.exe

# installing the .NET dependencies
dotnet add package Moq
dotnet add package NUnit
dotnet add package Microsoft.NET.Test.Sdk
dotnet add package NUnit3TestAdapter
dotnet add package coverlet.collector

# Running the command
dotnet test # Running the dotnet unit tests
dotnet test --collect:"XPlat Code Coverage" # Code Coverage

# Alternative Approach
https://www.youtube.com/watch?v=-bmnmmnC_qU

In [34]:
## Run all the setup required to run this code
%pip install llama-index
%pip install llama-index.llms-ollama
%pip install nest_asyncio
%pip install tree_sitter
%pip install tree_sitter_language_pack
%pip install llama-index-embeddings-huggingface 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
import shutil
from datetime import datetime
import xml.etree.ElementTree as ET

# Test Directory Path
TEST_DIRECTORY = "C:\\GitHub\\microservices-sample\\microservices\\tests"

target_source_list = [
    "C:\\GitHub\\microservices-sample\\microservices\\src\microservices\\CartMicroservice\\Controllers\\CartController.cs"
]


testcases = os.listdir(TEST_DIRECTORY)

#create a folder for this specific test run, where all the results will be achived
#testrun_folder = f"TestRun-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
#os.makedirs(testrun_folder, exist_ok=True)


def cleanup_testrun( testdir, testlist, result_folder="TestResults" ):

    ## before the start of the test, remove all 'TestResults' folder 
    for testcase in testlist:
        test_result_path = os.path.join(testdir, testcase, 'TestResults')
        if os.path.exists(test_result_path):
            shutil.rmtree(test_result_path)

cleanup_testrun( TEST_DIRECTORY, testcases, result_folder="TestResults" )

for testcase in testcases:
    
    testcase_dir = os.path.join(TEST_DIRECTORY, testcase)
    os.system(f'dotnet test --collect:"XPlat Code Coverage" "{testcase_dir}"')

    coverage_found = False
    for root, dirs, files in os.walk(testcase_dir):
        if 'coverage.cobertura.xml' in files:
            
            coverage_file_path = os.path.join(root, 'coverage.cobertura.xml')
            
            with open(coverage_file_path, 'r', encoding='utf-8') as f:
                coverage_xml_content = f.read()
            
            tree = ET.ElementTree(ET.fromstring(coverage_xml_content))
            
            root_elem = tree.getroot()
            line_rate = root_elem.attrib.get('line-rate')
            branch_rate = root_elem.attrib.get('branch-rate')
            
            print(f'testcase="{testcase}" coverage.line-rate="{line_rate}" coverage.branch-rate="{branch_rate}"')
            coverage_found = True
            break
    
    if not coverage_found:
        print('coverage.cobertura.xml file not found')




testcase="CartMicroservice.UnitTests" coverage.line-rate="0.0629" coverage.branch-rate="0"
testcase="CatalogMicroservice.UnitTests" coverage.line-rate="0.0823" coverage.branch-rate="0.07139999999999999"
testcase="IdentityMicroservice.UnitTests" coverage.line-rate="0.3343" coverage.branch-rate="0.475"


In [13]:
cleanup_testrun( TEST_DIRECTORY, testcases, result_folder="TestResults" )

In [2]:
## Import all llama-index libs here

from llama_index.core import  Settings, Document, VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole

import os

import nest_asyncio
nest_asyncio.apply()



c:\GitHub\unit-test-generator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# Initialize the value for Setting 
llm = Ollama( model="codestral", request_timeout=300)
#embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = llm

#Settings.embed_model = HuggingFaceEmbedding(
#    model_name="BAAI/bge-small-en-v1.5"
#)

#docs = SimpleDirectoryReader(
#    "C:\\GitHub\\microservices-sample\\microservices\\src\microservices\\CartMicroservice",
#    required_exts=[".cs"]
#).load_data()

target_source_list = [
    "C:\\GitHub\\microservices-sample\\microservices\\src\microservices\\CartMicroservice\\Controllers\\CartController.cs"
]

docs = []

for file_path in target_source_list:

    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
        docs.append(Document(text=content, metadata={"file_path": file_path}))

    index = VectorStoreIndex.from_documents(docs)

    message_templates = [
        ChatMessage(role=MessageRole.SYSTEM, content="You are an unit test expert who can analyse a file and create execution ready unit-test cases"),
        ChatMessage(role=MessageRole.USER, content=f"Analyse this source code {content} and create XUnit unit test cases based on the source code provided" )
    ]

    chat_prompt_template = ChatPromptTemplate(message_templates=message_templates)
    #chat_message = chat_template.format_messages(source_code=content)
    #chat_engine = index.as_chat_engine(verbose=False, ChatPromptTemplate=chat_template)

    response = Settings.llm.chat(  chat_prompt_template.format_messages() )
 
    #resposne = chat_engine.chat(message=f"generate XUnit test cases for the source code - {content}")  
    #response = chat_engine.chat( message="generate XUnit unit test cases")
    print(f'Response -> {response}')


In [ ]:
# Initialize the value for Setting 
Settings.llm  = Ollama( model="granite-code:20b", request_timeout=300)

relevant_sub_folders = [
    "Model",
    "Repository"
]

target_source_list = [
    "C:\\GitHub\\microservices-sample\\microservices\\src\\microservices\\CartMicroservice"
]

dependency_content = ""

for target_source in target_source_list:
    
    dependency_content = ""

    #within the target source, read all the important directories
    for sub_folder in relevant_sub_folders:
        sub_folder_path = os.path.join(target_source, sub_folder)
        if os.path.exists(sub_folder_path) and os.path.isdir(sub_folder_path):
            for fname in os.listdir(sub_folder_path):
                if fname.endswith('.cs'):
                    file_path = os.path.join(sub_folder_path, fname)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        file_content = f.read()
                        dependency_content += file_content + "\n---\n"

filepath = "C:\\GitHub\\microservices-sample\\microservices\\src\\microservices\\CartMicroservice\\Controllers\\CartController.cs"

with open(filepath, "r", encoding="utf-8") as f:
    controller_content = f.read()

message_templates = [
    ChatMessage(role=MessageRole.SYSTEM, content="You are an unit test expert who can analyse a file and create execution ready unit-test cases"),
    ChatMessage(role=MessageRole.ASSISTANT, content=f"Provide the source code" ),
    ChatMessage(role=MessageRole.USER, content=f"Here is the source code for which the XUnit test needs to be created {controller_content}" ),
    ChatMessage(role=MessageRole.ASSISTANT, content=f"Provide all dependencies in a single file, each dependency must be separated by '---' characters" ),
    ChatMessage(role=MessageRole.USER, content=f"Here are all the dependencies - {dependency_content}" ),
    ChatMessage(role=MessageRole.ASSISTANT, content=f"Any specific instructions before generation ?" ),
    ChatMessage(role=MessageRole.USER, content=f"Generate XUnit specific unit test cases for the source code provided, do not add additional classes" ),

]

chat_prompt_template = ChatPromptTemplate( message_templates=message_templates)
response = Settings.llm.chat(  chat_prompt_template.format_messages() )
print(f'Response -> {response}')

In [ ]:
# Initialize the value for Setting 
Settings.llm  = Ollama( model="llama4:scout", request_timeout=1000)

relevant_sub_folders = [
    "Model",
    "Repository"
]

target_source_list = [
    "C:\\GitHub\\microservices-sample\\microservices\\src\\microservices\\CartMicroservice"
]

dependency_content = ""

for target_source in target_source_list:
    
    dependency_content = ""

    #within the target source, read all the important directories
    for sub_folder in relevant_sub_folders:
        sub_folder_path = os.path.join(target_source, sub_folder)
        if os.path.exists(sub_folder_path) and os.path.isdir(sub_folder_path):
            for fname in os.listdir(sub_folder_path):
                if fname.endswith('.cs'):
                    file_path = os.path.join(sub_folder_path, fname)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        file_content = f.read()
                        dependency_content += file_content + "\n---\n"

filepath = "C:\\GitHub\\microservices-sample\\microservices\\src\\microservices\\CartMicroservice\\Controllers\\CartController.cs"

with open(filepath, "r", encoding="utf-8") as f:
    controller_content = f.read()

message_templates = [
    ChatMessage(role=MessageRole.SYSTEM, content="You are a developer who generates XUnit unit test cases"),
    ChatMessage(role=MessageRole.USER, content="Generate a blank template for XUnit unit-test case for csharp"),
    ChatMessage(role=MessageRole.USER, content=f"Generate positive unit test cases for the functions in the source code - {controller_content}"),
    ChatMessage(role=MessageRole.ASSISTANT, content=f"Share all dependencies for this class separated by --- characters"),
    ChatMessage(role=MessageRole.USER, content=f"Here are all the dependencies - {dependency_content} - make sure all references are included and the code compiles"),
    ChatMessage(role=MessageRole.USER, content=f"Make sure all variables are initialized with sample data")
]

chat_prompt_template = ChatPromptTemplate( message_templates=message_templates)
response = Settings.llm.chat(  chat_prompt_template.format_messages() )
print(f'Response -> {response}')